In [1]:
import sagemaker

# Create SageMaker session
sagemaker_session = sagemaker.Session()

# Data Bucket
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'

role = sagemaker.get_execution_role()

In [2]:
# Getting the MNIST Dataset
from torchvision import datasets, transforms

datasets.MNIST('data', download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

  0%|          | 16384/9912422 [00:00<01:13, 134955.24it/s]

9920512it [00:00, 19079028.10it/s]                           


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 510826.73it/s]
0it [00:00, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:01, 1200316.54it/s]                             
8192it [00:00, 259934.78it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: data
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.1307,), std=(0.3081,))
                         )
    Target Transforms (if any): None

In [3]:
# Upload dataset to S3 bucket
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-356045791542/sagemaker/DEMO-pytorch-mnist


In [4]:
# Training Script
!pygmentize mnist.py

import argparse
import json
import logging
import os
import sagemaker_containers
import sys
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)

In [5]:
# Run training in SageMaker
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mnist.py',
                    role=role,
                    framework_version='1.1.0',
                    train_instance_count=2,
                    train_instance_type='ml.c4.xlarge',
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo'
                    })

In [6]:
estimator.fit({'training': inputs})

2019-10-07 19:46:43 Starting - Starting the training job...
2019-10-07 19:46:46 Starting - Launching requested ML instances......
2019-10-07 19:47:51 Starting - Preparing the instances for training......
2019-10-07 19:49:04 Downloading - Downloading input data...
2019-10-07 19:49:36 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-10-07 19:49:32,360 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-10-07 19:49:32,362 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-10-07 19:49:32,374 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-10-07 19:49:38,599 sagemaker_pytorch_container.training INFO     Invoking user training script.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2

Train Epoch: 1 [19200/30000 (64%)] Loss: 0.934797
Train Epoch: 1 [19200/30000 (64%)] Loss: 0.913063
Train Epoch: 1 [25600/30000 (85%)] Loss: 0.837450
Train Epoch: 1 [25600/30000 (85%)] Loss: 0.671311
Test set: Average loss: 0.3247, Accuracy: 9098/10000 (91%)

Test set: Average loss: 0.3247, Accuracy: 9098/10000 (91%)

Train Epoch: 2 [6400/30000 (21%)] Loss: 0.544159
Train Epoch: 2 [6400/30000 (21%)] Loss: 0.494016
Train Epoch: 2 [12800/30000 (43%)] Loss: 0.470556
Train Epoch: 2 [12800/30000 (43%)] Loss: 0.382452
Train Epoch: 2 [19200/30000 (64%)] Loss: 0.673172
Train Epoch: 2 [19200/30000 (64%)] Loss: 0.543181
Train Epoch: 2 [25600/30000 (85%)] Loss: 0.644568
Train Epoch: 2 [25600/30000 (85%)] Loss: 0.498440
Test set: Average loss: 0.1927, Accuracy: 9423/10000 (94%)

Test set: Average loss: 0.1927, Accuracy: 9423/10000 (94%)

Train Epoch: 3 [6400/30000 (21%)] Loss: 0.333466
Train Epoch: 3 [6400/30000 (21%)] Loss: 0.483074
Train Epoch: 3 [12800/30000 (43%)] Loss: 0.338210
Train Epoch: 3

In [7]:
# Deploy the model to an endpoint for hosting
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

---------------------------------------------------------------------------------------!

In [10]:
from IPython.display import HTML
HTML(open("index.html").read())

In [13]:
import numpy as np

image = np.array([data], dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

5
